In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import datetime

In [2]:
base = 'https://www.resultats-elections.interieur.gouv.fr/telechargements/LG2024'

In [3]:
circonscriptions = pd.read_csv('lg2024_circonscriptions.csv')

In [4]:
mapping_dept = (
    pd
    .read_csv('lg2022_circonscriptions.csv')
    .groupby(['CodDpt3Car', 'CodDpt'])
    .agg({'CodCirLg': len})
    .reset_index()
    .set_index('CodDpt3Car')
    [['CodDpt']]
    .query('CodDpt.str.startswith("Z")')
    .to_dict()
    ['CodDpt']
)

mapping_dept

{'099': 'ZZ',
 '971': 'ZA',
 '972': 'ZB',
 '973': 'ZC',
 '974': 'ZD',
 '975': 'ZS',
 '976': 'ZM',
 '977': 'ZX',
 '986': 'ZW',
 '987': 'ZP',
 '988': 'ZN'}

In [5]:
t1_resultats = (
    pd
    .read_csv('lg2024/t1_resultats.csv')
)

t1_resultats

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,LibCirElec,NbSap,CodReg,CodDpt,CodCirc2
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,1184,"43,09","23,36",QUALIF T2,97501,975,0,Saint-Pierre-et-Miquelon,1,0,975,ZS001
1,2,BEAUMONT,Frédéric,M.,SOC,Parti socialiste,464,"16,89","9,15",QUALIF T2,97501,975,0,Saint-Pierre-et-Miquelon,1,0,975,ZS001
2,3,CHAGNON,Patricia,Mme,RN,Rassemblement National,291,"10,59","5,74",NON,97501,975,0,Saint-Pierre-et-Miquelon,1,0,975,ZS001
3,4,LETOURNEL,Marion,Mme,FI,La France insoumise,409,"14,88","8,07",NON,97501,975,0,Saint-Pierre-et-Miquelon,1,0,975,ZS001
4,5,LEBAILLY,Patrick,M.,DVG,Divers gauche,400,"14,56","7,89",NON,97501,975,0,Saint-Pierre-et-Miquelon,1,0,975,ZS001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,4,RONGIONE,Viviane,Mme,EXG,Extrême gauche,280,"0,63","0,41",NON,2B02,2B,94,2ème circonscription,1,94,2B,2B002
4005,5,GIACOMI,Jean-Antoine,M.,REG,Régionaliste,0,"0,00","0,00",NON,2B02,2B,94,2ème circonscription,1,94,2B,2B002
4006,6,JOUART,Sylvie,Mme,RN,Rassemblement National,11275,"25,42","16,60",QUALIF T2,2B02,2B,94,2ème circonscription,1,94,2B,2B002
4007,7,CARLI,Antò,M.,REG,Régionaliste,2277,"5,13","3,35",NON,2B02,2B,94,2ème circonscription,1,94,2B,2B002


In [6]:
nuance_to_groupe = {
    'UG' : 'Nouveau Front populaire',
    'EXG': 'Divers gauche',
    'COM': 'Divers gauche',
    'FI' : 'Divers gauche',
    'SOC': 'Divers gauche',
    'RDG': 'Divers gauche', 
    'VEC': 'Divers gauche', 
    'DVG': 'Divers gauche',
    'ECO': 'Divers gauche',
    'ENS': 'Ensemble',
    'REN': 'Ensemble',
    'MDM': 'Ensemble',
    'HOR': 'Ensemble',
    'UDI': 'Ensemble',
    'DVC': 'Divers centre',
    'LR' : 'Les Républicains',
    'DVD': 'Divers droite',
    'REC': 'Divers droite',
    'DSV': 'Divers droite',
    'EXD': 'Divers droite',
    'RN' : 'Rassemblement national',
    'UXD': 'Rassemblement national-Les Républicains',
    'REG': 'Régionalistes',
    'DIV': 'Autres'
    
}

t1_entete = (
    t1_resultats
    .assign(
        NbVoix = lambda df: df.NbVoix.astype(int)
    )
    .sort_values('NbVoix', ascending=False)
    .drop_duplicates('CodCirc2', keep='first')
    .set_index('CodCirc2')
    .sort_index()
    .assign(
        GrpLib = lambda df: df.CodNuaCand.apply(lambda x: nuance_to_groupe[x]),
        CandLib = lambda df: df.PrenomPsn+" "+df.NomPsn
    )
)

t1_entete.tail(30)

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,LibCirElec,NbSap,CodReg,CodDpt,GrpLib,CandLib
CodCirc2,,,,,,,,,,,,,,,,,,,
ZC001,1,CASTOR,Jean-Victor,M.,REG,Régionaliste,12895,"62,78","21,77",QUALIF T2,97301,973,3,1ère circonscription,1,3,973,Régionalistes,Jean-Victor CASTOR
ZC002,5,RIMANE,Davy,M.,REG,Régionaliste,8307,"60,21","16,73",QUALIF T2,97302,973,3,2ème circonscription,1,3,973,Régionalistes,Davy RIMANE
ZD001,1,NAILLET,Philippe,M.,UG,Union de la gauche,19186,"46,25","21,67",QUALIF T2,97401,974,4,1ère circonscription,1,4,974,Nouveau Front populaire,Philippe NAILLET
ZD002,5,LEBON,Karine,Mme,UG,Union de la gauche,19068,"47,72","18,74",QUALIF T2,97402,974,4,2ème circonscription,1,4,974,Nouveau Front populaire,Karine LEBON
ZD003,1,RIVIERE,Joseph,M.,RN,Rassemblement National,13360,"31,56","13,52",QUALIF T2,97403,974,4,3ème circonscription,1,4,974,Rassemblement national,Joseph RIVIERE
ZD004,2,KBIDI,Emeline,Mme,DVG,Divers gauche,22696,"42,28","20,36",QUALIF T2,97404,974,4,4ème circonscription,1,4,974,Divers gauche,Emeline KBIDI
ZD005,5,RATENON,Jean-Hugues,M.,DVG,Divers gauche,11536,"33,18","12,73",QUALIF T2,97405,974,4,5ème circonscription,1,4,974,Divers gauche,Jean-Hugues RATENON
ZD006,3,MAILLOT,Frédéric,M.,DVG,Divers gauche,10818,"29,76","12,76",QUALIF T2,97406,974,4,6ème circonscription,1,4,974,Divers gauche,Frédéric MAILLOT
ZD007,9,GAILLARD,Perceval,M.,UG,Union de la gauche,14851,"29,55","12,26",QUALIF T2,97407,974,4,7ème circonscription,1,4,974,Nouveau Front populaire,Perceval GAILLARD


In [7]:
lg2022_resultats = (
    pd
    .read_csv('lg2022_t2_resultats.csv')
    .assign(
        NbVoix = lambda df: df.NbVoix.astype(int)
    )
    .sort_values('NbVoix', ascending=False)
    .drop_duplicates('CodCirc2', keep='first')
    .pipe(lambda df: 
        pd.concat([
            df,
            pd
            .read_csv('lg2022_t1_resultats.csv')
            .query('Elu == "oui"')
        ])
    )
    .set_index('CodCirc2')
    .sort_index()
)

lg2022_resultats

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNua,LibNua,NbVoix,RapportExprime,RapportInscrit,Elu,CodCirc,CodCirc1
CodCirc2,,,,,,,,,,,,
01001,3,BRETON,Xavier,M.,LR,Les Républicains,24408,"63,22","28,32",oui,00101,00101
01002,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),24960,"58,34","24,97",oui,00102,00102
01003,3,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),18398,"58,72","22,38",oui,00103,00103
01004,4,BUISSON,Jérôme,M.,RN,Rassemblement National,22601,"62,27","23,81",oui,00104,00104
01005,4,ABAD,Damien,M.,DVD,Divers droite,17687,"57,86","22,83",oui,00105,00105
...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,2,PETIT,Frédéric,M.,ENS,Ensemble ! (Majorité présidentielle),23191,"60,21","18,79",oui,09907,09907
ZZ008,8,HABIB,Meyer,M.,UDI,Union des Démocrates et des Indépendants,8470,"50,58","6,46",oui,09908,09908
ZZ009,3,BEN CHEÏKH,Karim,M.,NUP,Nouvelle union populaire écologique et sociale,11348,"54,07","9,41",oui,09909,09909


In [8]:
def get_resultats(CodCirc, CodDept, CodeReg, tour):
    url = f'{base}/resultatsT{tour}/{CodDept}/R{tour}{CodCirc}.xml'
    
    try:
        df = (
            pd
            .read_xml(
                url,
                xpath=f'.//Tour[./NumTour = {tour}]//Candidat',
                dtype = 'object'
            )
            .assign(
                CodCirc=CodCirc,
                CodDept=CodDept,
                CodeReg=CodeReg
            )
        )
        #print(url)
    except:
        #print(f'error: {circonscription}')
        df = None
        
    
    return df
    
    
resultats_t2 = (
    pd
    .concat([
        get_resultats(c['CodCirElec'], c['CodDpt'], c['CodReg'], 2)
        for idx, c in tqdm(list(circonscriptions.reset_index().iterrows()))
    ])
    .join(
        circonscriptions
        .reset_index()
        .assign(
            CodCirc2 = lambda df: (
                df.CodDpt.replace(mapping_dept)
                + df.CodCirElec.astype(str).str[-2:].str.pad(3, fillchar='0')
            )
        )
        .set_index('CodCirElec'), #[['CodCirc2']],
        on = 'CodCirc'
    )
    .query('Elu == "OUI"')
)
resultats_t2

  0%|          | 0/577 [00:00<?, ?it/s]

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,index,LibCirElec,NbSap,CodReg,CodDpt,CodCirc2
0,1,LENORMAND,Stéphane,M.,DVD,Divers droite,1665,"61,74","32,85",OUI,97501,975,0,0,Saint-Pierre-et-Miquelon,1,0,975,ZS001
0,1,SANQUER,Nicole,Mme,DVD,Divers droite,17838,"55,88","25,76",OUI,98702,987,0,3,2ème circonscription,1,0,987,ZP002
0,3,REID ABERLOT,Mereana,Mme,UG,Union de la gauche,17308,"50,87","25,07",OUI,98703,987,0,4,3ème circonscription,1,0,987,ZP003
0,2,METZDORF,Nicolas,M.,DVD,Divers droite,34577,"52,41","35,59",OUI,98801,988,0,5,1ere circonscription,1,0,988,ZN001
0,2,TJIBAOU,Emmanuel,M.,REG,Régionaliste,51724,"57,44","41,40",OUI,98802,988,0,6,2ème circonscription,1,0,988,ZN002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,RIMBERT,Catherine,Mme,RN,Rassemblement National,29667,"55,39","34,92",OUI,8405,84,93,572,5ème circonscription,1,93,84,84005
0,7,MARCANGELI,Laurent,M.,ENS,Ensemble ! (Majorité présidentielle),20892,"63,20","39,80",OUI,2A01,2A,94,573,1ère circonscription,1,94,2A,2A001
0,1,COLOMBANI,Paul-André,M.,REG,Régionaliste,23969,"59,21","38,35",OUI,2A02,2A,94,574,2ème circonscription,1,94,2A,2A002
0,2,CASTELLANI,Michel,M.,REG,Régionaliste,24667,"64,33","39,71",OUI,2B01,2B,94,575,1ère circonscription,1,94,2B,2B001


In [9]:
t2_resultats = (
    resultats_t2
    .assign(
        NbVoix = lambda df: df.NbVoix.astype(int)
    )
    .sort_values('NbVoix', ascending=False)
    .drop_duplicates('CodCirc2', keep='first')
    .set_index('CodCirc2')
    .sort_index()
)

t2_resultats

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,index,LibCirElec,NbSap,CodReg,CodDpt
CodCirc2,,,,,,,,,,,,,,,,,,
01001,3,BRETON,Xavier,M.,LR,Les Républicains,33889,"56,48","39,02",OUI,0101,01,84,467,1ère circonscription,1,84,01
01002,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),38973,"55,09","38,24",OUI,0102,01,84,468,2ème circonscription,1,84,01
01003,1,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),32958,"63,00","39,16",OUI,0103,01,84,469,3ème circonscription,1,84,01
01004,6,BUISSON,Jérôme,M.,RN,Rassemblement National,33186,"51,35","34,52",OUI,0104,01,84,470,4ème circonscription,1,84,01
01005,8,CHAVENT,Marc,M.,UXD,Union de l'extrême droite,27040,"57,53","34,70",OUI,0105,01,84,471,5ème circonscription,1,84,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,1,PETIT,Frédéric,M.,ENS,Ensemble ! (Majorité présidentielle),34516,"58,21","26,39",OUI,ZZ07,ZZ,0,14,7ème circonscription,1,0,ZZ
ZZ008,6,YADAN,Caroline,Mme,ENS,Ensemble ! (Majorité présidentielle),18302,"52,70","12,29",OUI,ZZ08,ZZ,0,15,8ème circonscription,1,0,ZZ
ZZ009,9,BEN CHEÏKH,Karim,M.,UG,Union de la gauche,26271,"74,71","20,15",OUI,ZZ09,ZZ,0,16,9ème circonscription,1,0,ZZ


In [10]:
t2_resultats.to_csv('tmp/t2_live.csv')

In [11]:
final_resultats = (
    pd
    .concat([
        t2_resultats
        .reset_index()
        .assign(
            Elu2 = lambda df: df.Elu
        ),
        t1_resultats
        .query('Elu == "OUI"')
        .assign(
            Elu1 = lambda df: df.Elu
        )
    ])
    .set_index('CodCirc2')
    .sort_index()
)

final_resultats

,NumPanneauCand,NomPsn,PrenomPsn,CivilitePsn,CodNuaCand,LibNuaCand,NbVoix,RapportExprimes,RapportInscrits,Elu,CodCirc,CodDept,CodeReg,index,LibCirElec,NbSap,CodReg,CodDpt,Elu2,Elu1
CodCirc2,,,,,,,,,,,,,,,,,,,,
01001,3,BRETON,Xavier,M.,LR,Les Républicains,33889,"56,48","39,02",OUI,0101,01,84,467.0,1ère circonscription,1,84,01,OUI,NaN
01002,2,DAUBIÉ,Romain,M.,ENS,Ensemble ! (Majorité présidentielle),38973,"55,09","38,24",OUI,0102,01,84,468.0,2ème circonscription,1,84,01,OUI,NaN
01003,1,GIVERNET,Olga,Mme,ENS,Ensemble ! (Majorité présidentielle),32958,"63,00","39,16",OUI,0103,01,84,469.0,3ème circonscription,1,84,01,OUI,NaN
01004,6,BUISSON,Jérôme,M.,RN,Rassemblement National,33186,"51,35","34,52",OUI,0104,01,84,470.0,4ème circonscription,1,84,01,OUI,NaN
01005,8,CHAVENT,Marc,M.,UXD,Union de l'extrême droite,27040,"57,53","34,70",OUI,0105,01,84,471.0,5ème circonscription,1,84,01,OUI,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZ007,1,PETIT,Frédéric,M.,ENS,Ensemble ! (Majorité présidentielle),34516,"58,21","26,39",OUI,ZZ07,ZZ,0,14.0,7ème circonscription,1,0,ZZ,OUI,NaN
ZZ008,6,YADAN,Caroline,Mme,ENS,Ensemble ! (Majorité présidentielle),18302,"52,70","12,29",OUI,ZZ08,ZZ,0,15.0,8ème circonscription,1,0,ZZ,OUI,NaN
ZZ009,9,BEN CHEÏKH,Karim,M.,UG,Union de la gauche,26271,"74,71","20,15",OUI,ZZ09,ZZ,0,16.0,9ème circonscription,1,0,ZZ,OUI,NaN


In [12]:
final_resultats.to_csv('lg2024/t2_resultats.csv')

## manquant

In [13]:
circonscriptions.loc[ ~circonscriptions.CodCirc2.isin(final_resultats.index)]

,CodCirElec,LibCirElec,NbSap,CodReg,CodDpt,CodCirc2


In [14]:
export = (
    circonscriptions
    .reset_index()
    .assign(
        CodCirc2 = lambda df: (
            df.CodDpt.replace(mapping_dept)
            + df.CodCirElec.astype(str).str[-2:].str.pad(3, fillchar='0')
        )
    )
    .set_index('CodCirc2')[[]]
    .sort_index()
    .join(
        final_resultats
        .assign(
            GrpLib = lambda df: df.CodNuaCand.apply(lambda x: nuance_to_groupe[x]),
            CandLib = lambda df: df.PrenomPsn+" "+df.NomPsn
        )
        [['CandLib', 'CodNuaCand', 'LibNuaCand', 'GrpLib', 'Elu1','Elu2']]
    )
    .assign(
        Elu = lambda df: df.Elu1.replace({ "QUALIF T2": "", "NON": "" })
    )
    .join(
        lg2022_resultats
        .assign(
            SortantLabel = lambda df: df.PrenomPsn+" "+df.NomPsn+" - "+df.LibNua
        )
        [['SortantLabel']]
    )
    .fillna('')
)

export

,CandLib,CodNuaCand,LibNuaCand,GrpLib,Elu1,Elu2,Elu,SortantLabel
CodCirc2,,,,,,,,
01001,Xavier BRETON,LR,Les Républicains,Les Républicains,,OUI,,Xavier BRETON - Les Républicains
01002,Romain DAUBIÉ,ENS,Ensemble ! (Majorité présidentielle),Ensemble,,OUI,,Romain DAUBIÉ - Ensemble ! (Majorité président...
01003,Olga GIVERNET,ENS,Ensemble ! (Majorité présidentielle),Ensemble,,OUI,,Olga GIVERNET - Ensemble ! (Majorité président...
01004,Jérôme BUISSON,RN,Rassemblement National,Rassemblement national,,OUI,,Jérôme BUISSON - Rassemblement National
01005,Marc CHAVENT,UXD,Union de l'extrême droite,Rassemblement national-Les Républicains,,OUI,,Damien ABAD - Divers droite
...,...,...,...,...,...,...,...,...
ZZ007,Frédéric PETIT,ENS,Ensemble ! (Majorité présidentielle),Ensemble,,OUI,,Frédéric PETIT - Ensemble ! (Majorité présiden...
ZZ008,Caroline YADAN,ENS,Ensemble ! (Majorité présidentielle),Ensemble,,OUI,,Meyer HABIB - Union des Démocrates et des Indé...
ZZ009,Karim BEN CHEÏKH,UG,Union de la gauche,Nouveau Front populaire,,OUI,,Karim BEN CHEÏKH - Nouvelle union populaire éc...


In [15]:
export.to_csv('tmp/lg2024_t2_flourish.csv')

In [16]:
elus = final_resultats.groupby('CodNuaCand').count()[['Elu1', 'Elu2']].assign(Total=lambda df: df.Elu1+df.Elu2)

elus

,Elu1,Elu2,Total
CodNuaCand,,,
DIV,0,1,1
DVC,0,6,6
DVD,2,25,27
DVG,0,12,12
ECO,0,1,1
ENS,2,148,150
EXD,1,0,1
HOR,0,6,6
LR,1,38,39


In [17]:
(
    pd.concat([
        (
            t1_resultats
            .query('CodCirc2.isin(@t1_resultats.query("Elu == \'OUI\'").CodCirc2.unique())')
        ),
        t2_resultats
    ])
    .groupby('CodNuaCand')
    [['NbVoix']]
    .sum()
    .pipe(lambda df: df.assign(RapportExprimes = lambda df2: df2.NbVoix / df.NbVoix.sum()))
    .join(
        elus
        [['Total']],
        how='inner'
    )
    .assign(
        PctSieges = lambda df: df.Total / elus['Total'].sum(),
        EcartVoixSieges = lambda df: df.PctSieges - df.RapportExprimes,
        PctSiegesFinal = lambda df: df.Total / 577
    )
    .style
    .format(
        {
            'RapportExprimes': '{:,.2%}'.format,
            'PctSieges': '{:,.2%}'.format,
            'PctSiegesFinal': '{:,.2%}'.format,
            'EcartVoixSieges': '{:,.2%}'.format,
        },
        thousands = " "
    )
)

,NbVoix,RapportExprimes,Total,PctSieges,EcartVoixSieges,PctSiegesFinal
CodNuaCand,,,,,,
DIV,37 496,0.20%,1,0.17%,-0.03%,0.17%
DVC,175 166,0.93%,6,1.04%,0.11%,1.04%
DVD,835 486,4.44%,27,4.68%,0.24%,4.68%
DVG,391 217,2.08%,12,2.08%,-0.00%,2.08%
ECO,70 695,0.38%,1,0.17%,-0.20%,0.17%
ENS,5 485 269,29.16%,150,26.00%,-3.17%,26.00%
EXD,24 267,0.13%,1,0.17%,0.04%,0.17%
HOR,187 374,1.00%,6,1.04%,0.04%,1.04%
LR,1 388 871,7.38%,39,6.76%,-0.63%,6.76%


In [18]:
datetime.datetime.now()

datetime.datetime(2024, 7, 8, 1, 53, 10, 680201)